# What do I mean by DSL Algebra?

When I say algebra, I don't think about implementing i.e. DSL for linear algebra in Python. I think about algebra in general as in "a set of objects and a collection of operations on them".

I like algebras, when they are used in programming. Probably the most well-known is Relational Algebra. The object in Relational Algebra is a *relation*, with operations like *join*, *projection* and *union*. The nice thing about having something that works like an algebra is, that you always work with the same type of object, reusing the box of tools without breaking the flow.

That is one of the reasons I like Pythons collections. Pythons sets and list actually have quite a nice algebra built around them.


In [5]:
{1,2,3} | {3,4,5}

{1, 2, 3, 4, 5}

In [6]:
{1,2,3} & {3,4,}

{3}

In [7]:
{1,2,3} - {3,4,5}

{1, 2}

And what I really like is, that all the laws I remember about sets from discrete math still work. For example, with union as `|` and empty set as `set()`:

In [8]:
a = {1,2,3}
a == a|set()==set()|a


True

In [9]:
b = {3,4,5}
c = {5,6,7}
(a | b) | c == a | (b | c)

True

My personal preference is to use these math-like operators over calling methods on objects, even though methpd calls would produce the same result, and often would be more efficient

In [10]:
a = {1,2}
a.add(3)
a.add(4)
a.remove(2)
a == ({1,2} | {3} | {4}) - {2}

True

To be honest, I probably would write this code with method calls, because they are more readable than operators: 

In [11]:
a == {1,2,3}.union({3,4}).difference({2})

True

I don't really want to have operators everywhere, what I want is composability. Now-days we might call this *fluent* interface, but I want to go one step further, I want the interface to conform to predefined set of rules.

This is what I mean when I say algebra:
* interfaces I work with *compose* with each other, they interoperate well, they are *fluent*
* the implementation is *lawful*, it conforms to predefined rules

By **dsl algebra** I mean two distinct, but related things:
* making a dsl that forms an algebra
* creating algebra that helps with creating dsl-s

# Why?

So I have been watching too many talks from Scala conferences and reading too many posts from haskell enthusiasts.
And now I envy them. Because I am mostly working with javascript and python and barely anybody even knows what `monad` *is*, while on the scala-side they are using co-free co-monads to implement cool, extensible DSLs.

Ok, there is fantasy-land and Ramda, but last time my colleague was trying to show off Ramda to the rest of the team, he mostly got puzzled looks from the most of them. And there isn't much in Python.

But I still want to try all of those cool things I see in the typed functional languages.

# Monoids are cool

One of the most useful concept to use here is the one of a *monoid*. 
If we think of this as an interface that is implemented by a type *T*, it needs two things:
* the *empty* element
* the *append* operation, that takes any two T's and returns a new T

There are two laws:
* for any a of type T: append(a,empty) == append(empty,a) == a
* for any a,b,c of type T: append(a,append(b,c)) == append(append(a,b),c)

Now we have this interface, we can do cool things with this :)

And by cool thigs, I mean going through [this presentation](https://www.youtube.com/watch?v=WsA7GtUQeB8) by Gabriel Gonzalez and translating the concepts from haskell to python :)

If you are not scared of Haskell, I really recomend that presentation. There is even a [souce for it](https://github.com/Gabriel439/slides/tree/master/lambdaconf/category).

Typeclasses present the first translation hiccup. In haskell you use these to signidy that a type conforms to some interface, similarily to Java/C# interfaces. Nice thing about this is, that you can define implementations for existing types. This makes them more similiar to C# extension methods, or Clojure protocols. Fortunately, we have single dispatch in Python to simulate this.

In [12]:
from functools import singledispatch
@singledispatch
def mempty(a):
    raise Error("Not implemented for" + a)

@singledispatch
def mappend(a, b):
    raise Error("Not implemented for" + a)

In [13]:
@mempty.register(list)
def _(a):
    return []

@mappend.register(list)
def _(a,b):
    return a + b

In [14]:
mappend([1,2,3],[4,5,6])

[1, 2, 3, 4, 5, 6]

In [15]:
@mempty.register(None.__class__)
def _(a):
    return None

@mappend.register(None.__class__)
def _(a,b):
    return None

In [16]:
mappend(None,None) == None

True

# Generic functions

Now we can create a generic function that works on all monoids, such as mconcat, that takes a list and appends its contents.

In [17]:
def mconcat(l):
    acc = l[0]
    for x in l[1:]:
        acc = mappend(acc,x)
    return acc

mconcat([[1,2],[3,4,5]])

[1, 2, 3, 4, 5]

As you can see, we have run into our first problem. In theory we should be able to do mconcat of an [] and get the mempty for the lists member type. But python doesn't have typed lists. Well, we see how far will this get us :)

# Nesting

Second thing we could do, is to try to nest these inside of other structures.
For example, if you have n-tuple with monoids, you can prove, that the n-tuple is monoid. Well, in Python you can't implement the (), but there is algebraic thing that behavesnjuat like monoid without the mempty element, called semi-group, so lets talk about these instead :)

In [18]:

@mappend.register((0,0).__class__)
def _(a,b):
    return tuple(mappend(i,j) for i,j in zip(a,b))

In [19]:
mappend(([1,2,3],[10,11,12]),([4,5,6],[14,15]))

([1, 2, 3, 4, 5, 6], [10, 11, 12, 14, 15])

In [20]:
mappend(([1,2],([10,11],([20,21],[25,26]))),([3,4],([12,13],([22,23],[26,28]))))

([1, 2, 3, 4], ([10, 11, 12, 13], ([20, 21, 22, 23], [25, 26, 26, 28])))

In [21]:
@mappend.register("".__class__)
def _(a,b):
    return a+b

# Functions as semigroups

If we have function f that accepts type A as input param and returns type B, then f forms a semigroup if B forms a semigroup.

In [25]:
@mappend.register(mconcat.__class__)
def _(a,b):
  def result(*x):
    a_r= a(*x)
    b_r=b(*x)
    return mappend(a_r,b_r)
  return result

This means we can pipe a single arg to multiple functions that return None.

In [28]:
def phello(arg):
  print("Hello",arg)
  
def phi(arg):
  print("Hi",arg)

mappend(phello,phi)("World!")

Hello World!
Hi World!


Or they could return function that returns None :)

In [38]:
def promptName():
    name = input("Enter Your Name: ")
    return lambda: print("Hi ", name)

def promptAge():
    age = input("Enter Your Age: ")
    return lambda: print("Your Age is", age)

promptBoth = mappend(promptName,promptAge)

In [39]:
answers = promptBoth()

Enter Your Nameadam
Enter Your Age11


In [40]:
answers()

Hi  adam
Your Age is 11


We could actually get these back, if we return a monoid from tose inner functions.

In [41]:
def getName():
    name = input("Enter Your Name: ")
    return lambda: [name]

def getAge():
    age = input("Enter Your Age: ")
    return lambda: [age]

getBoth = mappend(getName,getAge)
listAnswers = getBoth()
listAnswers()

Enter Your Name: Adam
Enter Your Age: 11


['Adam', '11']

We could make a little library for asking for config from this :) First, lets create a mappend for Dictionary.

In [45]:
@mappend.register(dict)
def _(a,b):
    return {**a, **b}

Now we could create a generic function, i.e. askFor:

In [48]:
def askFor(name):
    def getAnswer():
        answer = input(name)
        return lambda: {name: answer}
    return getAnswer

In [49]:
askAll = mconcat([
        askFor('name'),
        askFor('age'),
        askFor('email')
    ])

In [50]:
allAnswers = askAll()

nameadam
age11
emaila@b.c


In [51]:
allAnswers()

{'age': '11', 'email': 'a@b.c', 'name': 'adam'}

To be honest, I mmight have over-done it with nesting functions:

In [52]:
def askFor(name):
    def getAnswer():
        answer = input(name)
        return {name: answer}
    return getAnswer

In [53]:
askAll = mconcat([
        askFor('name'),
        askFor('age'),
        askFor('email')
    ])

In [54]:
askAll()

namea
age11
emaila@b.c


{'age': '11', 'email': 'a@b.c', 'name': 'a'}

Now we could abuse the fact, that {} is the mempty in our little monoid, and return it if no prompt happens.

In [56]:
def askFor(name):
    def getAnswer():
        answer = input(name)
        if answer == "":
            return {}
        else: 
            return {name: answer}
    return getAnswer

In [57]:
askAll = mconcat([
        askFor('name'),
        askFor('age'),
        askFor('email')
    ])
askAll()

name
age11
email1@2.3


{'age': '11', 'email': '1@2.3'}

This allow us to easily add defaults :-)

In [59]:
def override(name,value):
    return lambda: {name:value}

askAll = mconcat([
        override('name','adam'),
        askFor('name'),
        askFor('age'),
        askFor('email')
    ])
askAll()

name
age11
email12


{'age': '11', 'email': '12', 'name': 'adam'}

What we could do now, instead of asking for input manually, we could pass in a config string and parse it.

In [63]:
import re
config = """
name:eve
age:16
dance:swing
"""

def parseFor(name):
    def getAnswer(config):
        m = re.search('(?<='+name+').*', config)
        if m == None:
            return {}
        else: 
            return {name: m.group(0)}
    return getAnswer

In [64]:
parseAll = mconcat([
        parseFor('name'),
        parseFor('age'),
        parseFor('email')
    ])

parseAll(config)

{'age': ':16', 'name': ':eve'}

And because we get the whole config, we could parse different things as well.

In [74]:
def getKeys(config):
    return {"keys": [x.strip() for x in re.split(":.*\n",config) if x.strip()!= ""]}

In [75]:
parseAll = mconcat([
        parseFor('name'),
        parseFor('age'),
        parseFor('email'),
        getKeys
    ])

parseAll(config)

{'age': ':16', 'keys': ['name', 'age', 'dance'], 'name': ':eve'}

In [71]:
[x.strip() for x in re.split(":.*\n",config) if x.strip()!= ""]

['name', 'age', 'dance']